Exploracion de los datos.

# Exploración de granularidad (sessionID) y categorías anómalas

Este notebook te muestra exactamente:
- **Duplicados / granularidad** en `contact_center` (múltiples filas por `sessionID` → necesidad de pivotar).
- **Categorías/respuestas** en `funnel_Q` y cómo quedaría un **preview del pivot**.
- **Metadatos/ruido** en `delitos_por_municipio.csv` (líneas crudas + parseo a partir de cabecera).
- **Valores atípicos** en `renta_por_hogar.csv` (`"."` en `Total`, tipos de datos).

In [ ]:
import pandas as pd
import chardet
from pathlib import Path
from io import StringIO
from IPython.display import display, Markdown

# Rutas (ajusta si es necesario)
p_renta = Path('data/renta_por_hogar.csv')
p_delitos = Path('data/delitos_por_municipio.csv')
p_contact = Path('data/contac_center_data.csv')

def detectar_encoding(path, n_bytes=20000):
    with open(path, "rb") as f:
        raw = f.read(n_bytes)
    return chardet.detect(raw)

def cargar_csv_flexible(path, enc):
    for sep in [';', ',', '\t']:
        try:
            df = pd.read_csv(path, sep=sep, encoding=enc, low_memory=False)
            return df, sep
        except Exception:
            continue
    return None, None


Detección de encodings

In [4]:
enc_renta = detectar_encoding(p_renta)
enc_delitos = detectar_encoding(p_delitos)
enc_contact = detectar_encoding(p_contact)

print("Encodings detectados:")
print("renta:", enc_renta)
print("delitos:", enc_delitos)
print("contact:", enc_contact)



Encodings detectados:
renta: {'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}
delitos: {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
contact: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


Explorar datset renta por hogar 

In [10]:
renta, sep_renta = cargar_csv_flexible(p_renta, enc_renta.get("encoding"))
delitos_raw, sep_delitos = cargar_csv_flexible(p_delitos, enc_delitos.get("encoding"))
contact, sep_contact = cargar_csv_flexible(p_contact, enc_contact.get("encoding"))

print("Separadores detectados:")
print("renta:", sep_renta, renta.shape if renta is not None else None)
print("delitos:", sep_delitos, delitos_raw.shape if delitos_raw is not None else None)
print("contact:", sep_contact, contact.shape if contact is not None else None)


Separadores detectados:
renta: ; (175068, 6)
delitos: None None
contact: ; (30997, 7)


Exploración de renta_por_hogar

In [11]:
display(Markdown("## renta_por_hogar.csv"))
display(renta.head(10))
print("Columnas:", renta.columns.tolist())
print("Valores únicos de 'Periodo':", renta["Periodo"].unique()[:10])
print("Número de '.' en Total:", (renta["Total"].astype(str) == ".").sum())


## renta_por_hogar.csv

,Municipios,Distritos,Secciones,Indicadores de renta media y mediana,Periodo,Total
0,"28001 Acebeda, La",NaN,NaN,Renta neta media por persona,2020,13.999
1,"28001 Acebeda, La",NaN,NaN,Renta neta media por persona,2019,.
2,"28001 Acebeda, La",NaN,NaN,Renta neta media por persona,2018,.
3,"28001 Acebeda, La",NaN,NaN,Renta neta media por persona,2017,.
4,"28001 Acebeda, La",NaN,NaN,Renta neta media por persona,2016,.
5,"28001 Acebeda, La",NaN,NaN,Renta neta media por persona,2015,.
6,"28001 Acebeda, La",NaN,NaN,Renta neta media por hogar,2020,27.408
7,"28001 Acebeda, La",NaN,NaN,Renta neta media por hogar,2019,.
8,"28001 Acebeda, La",NaN,NaN,Renta neta media por hogar,2018,.
9,"28001 Acebeda, La",NaN,NaN,Renta neta media por hogar,2017,.


Columnas: ['Municipios', 'Distritos', 'Secciones', 'Indicadores de renta media y mediana', 'Periodo', 'Total']
Valores únicos de 'Periodo': [2020 2019 2018 2017 2016 2015]
Número de '.' en Total: 1204


Exploración de delitos_por_municipio


In [21]:
display(Markdown("## delitos_por_municipio.csv (primeras líneas crudas)"))
with open(p_delitos, "r", encoding=enc_delitos.get("encoding"), errors="replace") as f:
    for i in range(20):
        print(next(f).rstrip())




## delitos_por_municipio.csv (primeras líneas crudas)


Balance de criminalidad. 2020 - 1er Trimestre - Capitales de provincia, municipios mayores de 30.000 habitantes e islas
Indicadores de seguridad 2020 - 1er Trimestre por geografía,  tipología penal y periodos.
Unidades: Hechos conocidos

;1.-Homicidios dolosos y asesinatos consumados;;2.-Homicidios dolosos y asesinatos en grado tentativa;;3.-Delitos graves y menos graves de lesiones y riña tumultuaria;;4.-Secuestro;;5.-Delitos contra la libertad e indemnidad sexual;;5.1.-Agresión sexual con penetración;;5.2.-Resto de delitos contra la libertad e indemnidad sexual;;6.-Robos con violencia e intimidación;;7.- Robos con fuerza en domicilios, establecimientos y otras instalaciones;;7.1.-Robos con fuerza en domicilios;;8.-Hurtos;;9.-Sustracciones de vehículos;;10.-Tráfico de drogas;;Resto de infracciones penales;;TOTAL INFRACCIONES PENALES;;
;Enero-marzo 2019;Enero-marzo 2020;Enero-marzo 2019;Enero-marzo 2020;Enero-marzo 2019;Enero-marzo 2020;Enero-marzo 2019;Enero-marzo 2020;Enero-marzo 20

In [ ]:
Exploración de contact_center

In [15]:
display(Markdown("### Vista general"))
display(contact.head(10))
display(Markdown("Columnas y tipos"))
display(pd.DataFrame({"columna": contact.columns, "dtype": [contact[c].dtype for c in contact.columns]}))

display(Markdown("### Frecuencia de filas por sessionID (top 15)"))
freq = contact["sessionID"].value_counts().head(15)
display(freq.to_frame("num_filas_por_sessionID"))

display(Markdown("### Muestra de categorías en `funnel_Q`"))
display(contact["funnel_Q"].astype(str).value_counts().head(20).to_frame("frecuencia"))

display(Markdown("### Vista ejemplo: filas de un sessionID con varias respuestas"))
if len(freq)>0:
    ejemplo = contact[contact["sessionID"]==freq.index[0]].head(10)
    display(ejemplo)

display(Markdown("### Preview de pivot (quick & dirty)"))
pivot_preview = (contact.assign(valor=1)
                 .pivot_table(index="sessionID", columns="funnel_Q", values="valor", aggfunc="max")
                 .fillna(0))
display(pivot_preview.iloc[:5, :10])


### Vista general

,sessionID,DNI,Telef,CP,duration_call_mins,funnel_Q,Producto
0,b'MC4zNTIyMjkyODczNzI4ODQ5NQ==',X61734151,632566551,28066,2.485994,Chalet,NaN
1,b'MC4zNTIyMjkyODczNzI4ODQ5NQ==',X61734151,632566551,28066,2.485994,Unifamiliar,NaN
2,b'MC4zNTIyMjkyODczNzI4ODQ5NQ==',X61734151,632566551,28066,2.485994,Sin Rejas,NaN
3,b'MC42NzQyNzE3NzczMDMyODM1',X61588112,646319047,28974,2.559412,Chalet,NaN
4,b'MC42NzQyNzE3NzczMDMyODM1',X61588112,646319047,28974,2.559412,Unifamiliar,NaN
5,b'MC42NzQyNzE3NzczMDMyODM1',X61588112,646319047,28974,2.559412,Sin Rejas,NaN
6,b'MC41OTQ2OTgwMzc0MDQ3MjM4',X67061561,666932439,28033,5.387563,Piso,NaN
7,b'MC41OTQ2OTgwMzc0MDQ3MjM4',X67061561,666932439,28033,5.387563,Bajo,NaN
8,b'MC41OTQ2OTgwMzc0MDQ3MjM4',X67061561,666932439,28033,5.387563,Con Rejas,NaN
9,b'MC41OTQ2OTgwMzc0MDQ3MjM4',X67061561,666932439,28033,5.387563,Sin Perro,NaN


Columnas y tipos

,columna,dtype
0,sessionID,object
1,DNI,object
2,Telef,int64
3,CP,int64
4,duration_call_mins,float64
5,funnel_Q,object
6,Producto,object


### Frecuencia de filas por sessionID (top 15)

...

,frecuencia
funnel_Q,
Piso,5461
Bajo,3760
Sin Rejas,3729
Con Rejas,3649
Chalet,3618
Sin Perro,2735
Con Perro,2726
Adosado,1862
Unifamiliar,1756


### Vista ejemplo: filas de un sessionID con varias respuestas

,sessionID,DNI,Telef,CP,duration_call_mins,funnel_Q,Producto
10188,b'MC43ODY4Mjc0MzA5NTEyMTQ0',X68621980,697352782,28911,8.694171,Chalet,NaN
10189,b'MC43ODY4Mjc0MzA5NTEyMTQ0',X68621980,697352782,28911,8.694171,Adosado,NaN
10190,b'MC43ODY4Mjc0MzA5NTEyMTQ0',X68621980,697352782,28911,8.694171,Con Rejas,NaN
10191,b'MC43ODY4Mjc0MzA5NTEyMTQ0',X68621980,697352782,28911,8.694171,Chalet,NaN
10192,b'MC43ODY4Mjc0MzA5NTEyMTQ0',X68621980,697352782,28911,8.694171,Adosado,NaN
10193,b'MC43ODY4Mjc0MzA5NTEyMTQ0',X68621980,697352782,28911,8.694171,Con Rejas,NaN


### Preview de pivot (quick & dirty)

funnel_Q,Adosado,Bajo,Chalet,Con Perro,Con Rejas,Intermedio,Piso,Sin Perro,Sin Rejas,Unifamiliar
sessionID,,,,,,,,,,
b'MC40MDA1NDM0MzY5MzYyODE4',0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
b'MC40MDA2OTkxNDAyNzA5Nzcz',1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
b'MC40MDA5Mzc0NTc5MzM5OTAxNg==',0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
b'MC40MDA5MzgwNDg5NzU2ODM2NA==',0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
b'MC40MDA5NjA1MDcwNjY2MzQ1NQ==',0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0


Al ejecutar cada celda en Jupyter:

Encoding correcto: Se verifico con chardet y al mostrar las cabeceras (renta, contact) los acentos aparecían bien.

Separador correcto: En renta y contact salió ;, en delitos trae ruido y había que saltar líneas + usar ;.

Tipos de datos: Total en renta estaba como string con ".", y que CP estaba cargado como número cuando debería ser string.

Valores faltantes : los "." en renta, y los NaN en Producto del contact center.

Duplicados: el análisis de sessionID mostró que un mismo ID tiene varias filas (una por respuesta), lo cual confirma que hay que pivotar.

Valores anómalos en categorías → en delitos se ven encabezados basura mezclados con datos, y en contact_center hay respuestas tipo "Sin Rejas", "Chalet", "Unifamiliar" que deben reorganizarse en columnas.